<a href="https://colab.research.google.com/github/hscrown/bigdata/blob/main/%EB%AA%A8%EC%9D%98%EA%B3%A0%EC%82%AC_2%ED%9A%8C_2%EC%9C%A0%ED%98%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2. 데이터셋을 이용하여 파킨슨병을 예측하는 모델을 로지스틱 회귀모형을 적용하여 생성하고, 이때 파킨슨병을 예측하는데 영향을 미치는 변수를 중요한 순서대로 3개 선정하시오.

이 모델에서 파킨슨병으로 진단하는 기준(threshold 또는 cutoff)을 0.5로 했을 때와 0.8로 했을 때의 F1-score를 비교하고 해석하시오

분석 조건
- 필요없는 컬럼 'name' 제거
- 로지스틱 회귀를 위해 상수항 추가
- 트레이닝셋과 테스트셋 비율 9:1
- 모델의 최적화 방법론 "bfgs"사용
- 데이터 정규화는 min-max 스케일러 사용
- status는 카테고리 타입으로 변환
- 모델은 로지스틱 회귀분석 사용

로지스틱 회귀는 이진형(0또는1) 형태를 갖는 종속변수(분류문제)에 대해 회귀식의 형태로 모형을 추정하는 것이다.

회귀계수의 계산은 최대우도추정법(MLE)를 따른다.

데이터 설명
- 환자들의 뇌를 촬영한 사진의 상태를 기록한 자료에 각 환자의 상테를 status 1: 파킨슨병 진단 , 0: 파킨슨병 아님 으로 추가한 테이블 이다.

분석흐름
- 데이터 로딩 > 데이터 전처리 > 모델 생성과 변수 선정 > 임계치에 따른 모델 정확도 비교

In [7]:
# 데이터 로딩
import pandas as pd
import numpy as np

# 머신러닝 알고리즘 및 평가
import statsmodels.api as sm
# 데이터 스플릿 하는 함수 임포트
from sklearn.model_selection import train_test_split

import sklearn.preprocessing as preprocessing
from sklearn import metrics
from sklearn.metrics import f1_score

In [3]:
dat = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/dataset/main/Parkinsons.csv")

In [4]:
dat.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [5]:
# 필요없는 컬럼 name 삭제
dat_preprocessing = dat.drop('name', axis=1, inplace=False)

In [8]:
# Normalization
dat_preprocessing.norm = preprocessing.minmax_scale(dat_preprocessing)
dat_processed = pd.DataFrame(dat_preprocessing.norm)
dat_processed.columns = dat_preprocessing.columns

<ipython-input-8-e30d3d22c2a1>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  dat_preprocessing.norm = preprocessing.minmax_scale(dat_preprocessing)


In [9]:
# 상수항 추가 : sm.add_constant()
dat_processed = sm.add_constant(dat_processed, has_constant='add')
dat_processed.head()

,const,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,1.0,0.184308,0.112592,0.054815,0.195680,0.249012,0.145472,0.247588,0.145288,0.312215,...,0.332584,0.068307,0.511745,1.0,0.369155,0.960148,0.569875,0.585765,0.390661,0.497310
1,1.0,0.198327,0.094930,0.278323,0.254130,0.288538,0.191233,0.323687,0.191042,0.472887,...,0.516048,0.059331,0.432577,1.0,0.470830,0.977024,0.703277,0.741337,0.473145,0.671326
2,1.0,0.165039,0.059128,0.265288,0.280178,0.328063,0.229287,0.369239,0.229411,0.390634,...,0.443317,0.039596,0.496220,1.0,0.404416,1.000000,0.636745,0.686371,0.408819,0.596682
3,1.0,0.165004,0.072927,0.264200,0.263342,0.328063,0.209056,0.324759,0.208862,0.414278,...,0.475478,0.040997,0.495936,1.0,0.416255,0.975885,0.695627,0.738089,0.436977,0.671949
4,1.0,0.161150,0.080909,0.260107,0.354511,0.407115,0.282755,0.437299,0.282870,0.499452,...,0.584542,0.054174,0.455499,1.0,0.375159,0.992813,0.762472,0.513798,0.404336,0.757611


In [10]:
# 독립변수와 종속변수 분리
feature_columns = list(dat_processed.columns.difference(['status']))
X = dat_processed[feature_columns]
y = dat_processed['status'].astype('category')

In [11]:
# 데이터셋 분리 9:1
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.1, random_state=2017010500, stratify=y)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((175, 23), (20, 23), (175,), (20,))

In [12]:
# 모델 생성과 변수 선정(statsmodel.api 패키지의 sm.Logit()함수로 로지스틱 회귀를 생성)
## 모델의 결과 p-value를 생성해주기 때문에 변수의 중요도를 고려해야할때 사용하면 좋다
model = sm.Logit(train_y, train_x)
result = model.fit(method='bfgs', max_iter=1000)
print(result.summary()) # p-value가 작을수록 종속변수의 예측에 영향을 미치는 정도가 높다고 판단. 작은 순서대로 3개를 선정

         Current function value: 0.281354
         Iterations: 35
         Function evaluations: 36
         Gradient evaluations: 36
                           Logit Regression Results                           
Dep. Variable:                 status   No. Observations:                  175
Model:                          Logit   Df Residuals:                      152
Method:                           MLE   Df Model:                           22
Date:                Fri, 15 Nov 2024   Pseudo R-squ.:                  0.4954
Time:                        14:37:16   Log-Likelihood:                -49.237
converged:                      False   LL-Null:                       -97.576
Covariance Type:            nonrobust   LLR p-value:                 2.442e-11
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
D2                   4.2639      3.099      1.376      0.169    

/usr/local/lib/python3.10/dist-packages/statsmodels/base/optimizer.py:19: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method bfgs is: gtol, norm, epsilon. The list of unsupported keyword arguments passed include: max_iter. After release 0.14, this will raise.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_optimize.py:1292: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(f, x0, args, fprime, callback=callback, **opts)
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [15]:
# 임계치에 따른 모델 정확도 비교
## 임계치에 따라 예측된 확률값을 1또는 0으로 분할하는 함수 작성
def cut_off(y,threshold):
    Y = y.copy()
    Y[Y>threshold] = 1
    Y[Y<=threshold] = 0
    return Y.astype(int)

In [16]:
# predict()함수를 이용하여 테스트셋에 대한 예측확률을 산출하고 임계치적용
test_y_pred_prob = result.predict(test_x)
test_y_pred = cut_off(test_y_pred_prob, 0.8)


In [17]:
# f1 score 산출
f1_score(test_y, test_y_pred) # 임계값이 0.8일때 0.78, 0.5일때 0.93

0.7142857142857143